## Exploring 311 Service Requests Data of NYC

The dataset is obtained from NYC Open Data and it contains all 311 Service Requests from 2010 to present. This information is automatically updated daily

### Team Members:
- Ritik Lnu
- Neel Shah
- Vishal Shah

We import the openclean library which helps immensely in cleaning the dataset.
It contains many functions such as outlier detectionl, clustering and removing missing values.

In [32]:
pip install openclean[full]

## Data Source
The data used in this project was obtained from the following sources: 

- [NYC Open Data's 311 Service Requests from 2010 to Present](https://data.cityofnewyork.us/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9)
  - This dataset contains information about the time, location, complaint type, and status of more than 24 million 311 service requests made in New York City within the past decade.

In [33]:
# Loading all the relevant libraries
import pandas as pd
import numpy as np
from openclean.profiling.dataset import dataset_profile
from openclean.cluster.key import KeyCollision
from openclean.function.value.key.fingerprint import Fingerprint
from openclean.operator.map.violations import fd_violations
from openclean.operator.collector.count import distinct
from openclean.function.eval.logic import And
from openclean.function.value.phonetic import Soundex, soundex
from openclean.pipeline import stream
from openclean.data.load import dataset
from openclean.cluster.key import KeyCollision
from openclean_geo.address.usstreet import USStreetNameKey
from openclean_geo.address.usstreet import StandardizeUSStreetName

We have used Google Colaboratory to clean this dataset and hence to access the data we first upload it to Google Drive and then access it from here by  mounting the Google Drive.

In [34]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### As we have 27M rows, we need to work on the subset of the dataset. So we have sampled our data into 5% of the full data which is around 1.395M rows.

In [35]:
#Please change path for your system
datafile = '311_Service_Requests_from_2010_to_Present.csv'

In [36]:
import pandas as pd
df = pd.read_csv(datafile)
df.head()

FileNotFoundError: ignored

In [ ]:
df_new = df.sample(frac=0.05, random_state=43)

In [ ]:
df_new.shape

In [ ]:
#Please change the path for your system
df_new.to_csv('Sampled_Data.csv', header=True, encoding="utf-8")

## Scrubbing the Data

In [38]:
# Loading dataset
#Please change the path for your system
original_df = pd.read_csv("Sampled_Data.csv",low_memory = False)

df = original_df.copy()
df.head()

,Unnamed: 0,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,Street Name,Cross Street 1,Cross Street 2,Intersection Street 1,Intersection Street 2,Address Type,City,Landmark,Facility Type,Status,Due Date,Resolution Description,Resolution Action Updated Date,Community Board,BBL,Borough,X Coordinate (State Plane),Y Coordinate (State Plane),Open Data Channel Type,Park Facility Name,Park Borough,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Latitude,Longitude,Location
0,1161152,31247536,08/05/2015 06:35:55 PM,08/19/2015 09:09:07 AM,DPR,Department of Parks and Recreation,Root/Sewer/Sidewalk Condition,Sidewalk Consultation,Street,11360.0,214-09 23 AVENUE,23 AVENUE,BELL BOULEVARD,215 STREET,NaN,NaN,ADDRESS,BAYSIDE,NaN,NaN,Closed,09/04/2015 06:35:55 PM,The agency has mailed literature to the custom...,08/19/2015 08:58:52 AM,07 QUEENS,4.059500e+09,QUEENS,1046185.0,224448.0,PHONE,Unspecified,QUEENS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.782517,-73.776357,"(40.782517127277416, -73.77635733957257)"
1,19331004,49902206,03/01/2021 11:22:05 AM,03/02/2021 05:31:24 PM,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,APARTMENT ONLY,RESIDENTIAL BUILDING,11691.0,51-24 BEACH CHANNEL DRIVE,BEACH CHANNEL DRIVE,NaN,NaN,NaN,NaN,ADDRESS,Far Rockaway,NaN,NaN,Closed,NaN,The Department of Housing Preservation and Dev...,03/02/2021 12:00:00 AM,14 QUEENS,4.160010e+09,QUEENS,1044756.0,156219.0,ONLINE,Unspecified,QUEENS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.595255,-73.782130,"(40.59525476424622, -73.78213009462443)"
2,20420950,26123456,08/14/2013 03:51:55 PM,08/21/2013 08:00:38 PM,DOF,Refunds and Adjustments,DOF Property - Payment Issue,Property Refunds and Credits,Property Address,11216.0,NaN,NaN,NaN,NaN,NaN,NaN,ADDRESS,BROOKLYN,NaN,NaN,Closed,08/24/2013 03:51:55 PM,See notes.,08/21/2013 08:00:10 PM,03 BROOKLYN,NaN,BROOKLYN,NaN,NaN,PHONE,Unspecified,BROOKLYN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,22013676,37085573,08/31/2017 02:40:08 PM,09/01/2017 12:55:00 PM,DOT,Department of Transportation,Street Condition,Pothole,NaN,11413.0,NaN,NaN,143 AVENUE,232 STREET,143 AVENUE,232 STREET,INTERSECTION,Springfield Gardens,NaN,NaN,Closed,NaN,The Department of Transportation inspected thi...,09/01/2017 12:55:00 PM,13 QUEENS,NaN,QUEENS,1055188.0,181556.0,UNKNOWN,Unspecified,QUEENS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.664722,-73.744300,"(40.66472182521037, -73.74430045066177)"
4,11790222,16279399,03/22/2010 02:46:00 PM,03/22/2010 05:30:00 PM,DOT,Department of Transportation,Traffic Signal Condition,Controller,NaN,NaN,NaN,NaN,NaN,NaN,NAGLE AVE,DYCKMAN ST,INTERSECTION,NaN,NaN,NaN,Closed,NaN,Service Request status for this request is ava...,03/22/2010 12:00:00 AM,Unspecified MANHATTAN,NaN,MANHATTAN,NaN,NaN,UNKNOWN,Unspecified,MANHATTAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
df.shape

(1353465, 42)

##Remove Duplicate Rows from the dataset


In [40]:
df = df.drop_duplicates()

In [41]:
df.shape

(1353465, 42)

## Data Profiling
In many data analytics projects, data profiling is an essential first step. Profiling allows users to understand the properties of data and to identify data quality problems. <b>openclean</b> allows users to generate metadata about the data at hand using a variety of different data profiling operators. openclean is designed to be extensible, to make it easy to add new functionality, and to customize data profiling and cleaning operators.<br><br>
We can make use of the default column profiler to compute statistics such as the number of distinct values in each column, the number of missing values, etc. These results are compiled into a dictionary. Using the stats() method, a summary of the results is accessible as a data frame.

In [ ]:
profiles = dataset_profile(df)
profiles.stats()

In [ ]:
print(profiles.types())

## Dealing with Missing and Superfluous Data
Let's check the column data types and identify columns with missing values greater than 50 % of the total rows in the dataset. We remove such columns as we cannot obtain any useful information from missing values in the dataset.

In [75]:
'''We Are Removing Some Coloumns that have more than 50% missing data And Unnecessary column'''

df.replace('Unspecified',np.nan,inplace=True)
columns_with_nan_values=df.isnull().sum() 
size = df.shape[0]
remove = []
for key,value in columns_with_nan_values.items():
    k = value/size 
    print(k,key) 
    if (value/size) > 0.5: 
        remove.append(key) 

df.drop(labels = remove, axis =1, inplace =True)

0.0 Unnamed: 0
0.0 Unique Key
0.0 Created Date
0.02059610760785737 Closed Date
0.0 Agency
0.0 Agency Name
0.0 Complaint Type
0.0 Descriptor
0.20924686165352135 Location Type
0.0 Incident Zip
0.18924375085061018 Incident Address
0.1892729146279026 Street Name
0.3559633443723631 Cross Street 1
0.35691278734421683 Cross Street 2
0.8367411747169493 Intersection Street 1
0.8367217321987543 Intersection Street 2
0.046425493029857226 Address Type
0.0 City
0.9656483107692109 Landmark
0.7068133064594526 Facility Type
3.2404196991594352e-06 Status
0.5037232422343342 Due Date
0.02226492375292448 Resolution Description
0.007796449796177601 Resolution Action Updated Date
0.0 Community Board
0.24172234787849722 BBL
0.0082598298131574 Borough
0.06668135656930285 X Coordinate (State Plane)
0.06668135656930285 Y Coordinate (State Plane)
0.0 Open Data Channel Type
0.9880558129888983 Park Facility Name
0.0082598298131574 Park Borough
0.999552822081516 Vehicle Type
0.9989565848568707 Taxi Company Borough


We remove the columns named 'Unnamed:0' and 'Unique Key' as they are only random sequences and cannot be used for our further analysis.

In [76]:
df = df.drop(["Unnamed: 0", "Unique Key"], axis = 1)

#### The dataset consists of float, integer, and object data types. Most of the columns that contain missing values appear to be associated with particular complaint types. Next, we overwrite the dataframe to include only the rows and columns of interest.

#### Below are the names of the columns with missing values that have been removed

In [42]:
set(original_df.columns) - set(df.columns)

set()

Below are the columns which are remaining after removing the rows with majority missing values and redundant columns.

In [43]:
df.columns

Index(['Unnamed: 0', 'Unique Key', 'Created Date', 'Closed Date', 'Agency',
       'Agency Name', 'Complaint Type', 'Descriptor', 'Location Type',
       'Incident Zip', 'Incident Address', 'Street Name', 'Cross Street 1',
       'Cross Street 2', 'Intersection Street 1', 'Intersection Street 2',
       'Address Type', 'City', 'Landmark', 'Facility Type', 'Status',
       'Due Date', 'Resolution Description', 'Resolution Action Updated Date',
       'Community Board', 'BBL', 'Borough', 'X Coordinate (State Plane)',
       'Y Coordinate (State Plane)', 'Open Data Channel Type',
       'Park Facility Name', 'Park Borough', 'Vehicle Type',
       'Taxi Company Borough', 'Taxi Pick Up Location', 'Bridge Highway Name',
       'Bridge Highway Direction', 'Road Ramp', 'Bridge Highway Segment',
       'Latitude', 'Longitude', 'Location'],
      dtype='object')

#### The 'Created_Date' and 'Closed Date' column currently contains string values so the pandas.to_datetime() method converts each string value to a datetime object below. This will be useful to us to know more about the data in temporal terms.

In [44]:
df['Created Date'] = pd.to_datetime(df['Created Date'])
df['Closed Date'] = pd.to_datetime(df['Closed Date'])
df['Resolution Action Updated Date'] = pd.to_datetime(df['Resolution Action Updated Date'])

KeyboardInterrupt: ignored

#### The values in the 'Agengy' & 'AgengyName' column also have inconsistent formatting: all but one have abbreviated names.

In [45]:
df["Agency"].unique()

array(['DPR', 'HPD', 'DOF', 'DOT', 'NYPD', 'DOHMH', 'DSNY', 'DEP', 'DOB',
       'TLC', 'DFTA', '3-1-1', 'DCA', 'DHS', 'HRA', 'EDC',
       'MAYORâ\x80\x99S OFFICE OF SPECIAL ENFORCEMENT', 'DOE', 'DOITT',
       'DCP', 'NYCEM', 'ACS', 'DCAS', 'DORIS', 'TAX', 'DVS', 'COIB',
       'FDNY'], dtype=object)

In [46]:
df["Agency Name"].unique()

array(['Department of Parks and Recreation',
       'Department of Housing Preservation and Development',
       'Refunds and Adjustments', 'Department of Transportation',
       'New York City Police Department',
       'Department of Health and Mental Hygiene', 'Brooklyn North 03',
       'Lot Cleaning', 'BCC - Queens West',
       'Department of Environmental Protection',
       'Personal Exemption Unit', 'Department of Buildings',
       'Taxi and Limousine Commission', 'Department of Sanitation',
       'Bronx 10', 'BCC - Brooklyn South', 'Department for the Aging',
       '3-1-1 Call Center', 'BCC - Bronx',
       'Department of Consumer Affairs', 'Property Exec Office',
       'Department of Homeless Services', 'A - Queens',
       'Brooklyn South 10', 'BCC - Brooklyn North',
       'HRA Benefit Card Replacement', 'DHS Advantage Programs',
       'Bronx 04',
       'A - Illegal Posting Staten Island, Queens and Brooklyn',
       'A - Illegal Posting Manhattan and Bronx', 'BCC - 

##### We'll convert the ''MAYOR'S OFFICE OF SPECIAL ENFORCEMENT" value to its abbreviated name, OSE for both the colummns named 'Agency' and 'Agency Name'

In [47]:
def update_OSE(agency):

  """Takes in an agency name and converts to abbreviated form, if the agency is
     the Office of Special Enforcement."Mayorâ\x80\x99s Office of Special Enforcement'"""

  if agency == 'MAYORâ\x80\x99S OFFICE OF SPECIAL ENFORCEMENT' or  agency == 'Mayorâ\x80\x99s Office of Special Enforcement' :
      return "OSE"
  else:
      return agency

In [48]:
df["Agency"] = df["Agency"].map(lambda agency: update_OSE(agency))
df["Agency Name"] = df["Agency Name"].map(lambda agency: update_OSE(agency))

In [49]:
df["Agency"].unique()

array(['DPR', 'HPD', 'DOF', 'DOT', 'NYPD', 'DOHMH', 'DSNY', 'DEP', 'DOB',
       'TLC', 'DFTA', '3-1-1', 'DCA', 'DHS', 'HRA', 'EDC', 'OSE', 'DOE',
       'DOITT', 'DCP', 'NYCEM', 'ACS', 'DCAS', 'DORIS', 'TAX', 'DVS',
       'COIB', 'FDNY'], dtype=object)

In [50]:
df["Agency Name"].unique()

array(['Department of Parks and Recreation',
       'Department of Housing Preservation and Development',
       'Refunds and Adjustments', 'Department of Transportation',
       'New York City Police Department',
       'Department of Health and Mental Hygiene', 'Brooklyn North 03',
       'Lot Cleaning', 'BCC - Queens West',
       'Department of Environmental Protection',
       'Personal Exemption Unit', 'Department of Buildings',
       'Taxi and Limousine Commission', 'Department of Sanitation',
       'Bronx 10', 'BCC - Brooklyn South', 'Department for the Aging',
       '3-1-1 Call Center', 'BCC - Bronx',
       'Department of Consumer Affairs', 'Property Exec Office',
       'Department of Homeless Services', 'A - Queens',
       'Brooklyn South 10', 'BCC - Brooklyn North',
       'HRA Benefit Card Replacement', 'DHS Advantage Programs',
       'Bronx 04',
       'A - Illegal Posting Staten Island, Queens and Brooklyn',
       'A - Illegal Posting Manhattan and Bronx', 'BCC - 

#### If we look at unique values in the 'complaint type' column, we'll see that some values are in title case while others are in upper case. Also some values contains a value with special characters.

In [51]:
df["Complaint Type"].unique()

array(['Root/Sewer/Sidewalk Condition', 'HEAT/HOT WATER',
       'DOF Property - Payment Issue', 'Street Condition',
       'Traffic Signal Condition', 'ELECTRIC', 'Noise - Residential',
       'Illegal Parking', 'Rodent', 'Sanitation Condition', 'Vacant Lot',
       'Street Light Condition', 'Standing Water', 'Snow', 'Sewer',
       'Food Establishment', 'DOF Property - Reduction Issue',
       'Water System', 'PLUMBING', 'Indoor Air Quality',
       'Blocked Driveway', 'WATER LEAK', 'Building/Use',
       'For Hire Vehicle Complaint',
       'Request Large Bulky Item Collection', 'Illegal Fireworks',
       'Noise - Street/Sidewalk', 'DOOR/WINDOW', 'New Tree Request',
       'OUTSIDE BUILDING', 'General Construction/Plumbing',
       'PAINT/PLASTER', 'Sidewalk Condition', 'Food Poisoning',
       'Missed Collection (All Materials)', 'Posting Advertisement',
       'Damaged Tree', 'Water Quality', 'Broken Muni Meter',
       'Elder Abuse', 'Traffic', 'Asbestos', 'HEATING', 'Noise Surv

In [52]:
df = df[~df['Complaint Type'].str.contains(r'[\.@*$]+')]

In [53]:
# We convert the values into title case
df['Complaint Type']=df['Complaint Type'].astype(str).apply(lambda x: x.capitalize())


#### Below, all of the unnecessry values in the column are removed.

In [54]:
df["Complaint Type"].unique()

array(['Root/sewer/sidewalk condition', 'Heat/hot water',
       'Dof property - payment issue', 'Street condition',
       'Traffic signal condition', 'Electric', 'Noise - residential',
       'Illegal parking', 'Rodent', 'Sanitation condition', 'Vacant lot',
       'Street light condition', 'Standing water', 'Snow', 'Sewer',
       'Food establishment', 'Dof property - reduction issue',
       'Water system', 'Plumbing', 'Indoor air quality',
       'Blocked driveway', 'Water leak', 'Building/use',
       'For hire vehicle complaint',
       'Request large bulky item collection', 'Illegal fireworks',
       'Noise - street/sidewalk', 'Door/window', 'New tree request',
       'Outside building', 'General construction/plumbing',
       'Paint/plaster', 'Sidewalk condition', 'Food poisoning',
       'Missed collection (all materials)', 'Posting advertisement',
       'Damaged tree', 'Water quality', 'Broken muni meter',
       'Elder abuse', 'Traffic', 'Asbestos', 'Heating', 'Noise surv

#### To numerically encode the "Descriptor" column in the modeling phase, we'll first need to exclude all values that aren't letters and we also need to remove the Null values. Below, an empty space is substituted for all numbers and non-letter symbols.

In [57]:
df['Descriptor'].isnull().sum()
df = df[df['Descriptor'].notna()]

In [58]:
print('The unique categories are: %d \n\nThey are the following: \n%s' % (len(df.Descriptor.unique()), df.Descriptor.unique()))

The unique categories are: 1483 

They are the following: 
['Sidewalk Consultation' 'APARTMENT ONLY' 'Property Refunds and Credits'
 ... 'Pool/Billiard Hall' 'Warning Buzzer' 'Debris']


In [59]:
import re
num_pattern = r'[-()0-9]'
df.Descriptor = df.Descriptor.map(lambda x: re.sub(num_pattern, '', x).lower())
df.Descriptor = df.Descriptor.map(lambda x: re.sub('/', ' ', x))
# We convert the values into title case
df['Descriptor']=df['Descriptor'].astype(str).apply(lambda x: x.capitalize())


#### Since Incident Zip is the most accurate value in the dataset because through that only we can know the area from where we get the most compaints. So, we are removing rows with Null values

In [60]:
df = df[df['Incident Zip'].notna()]

In [61]:
df["Incident Zip"].unique()

array(['11360.0', '11691.0', '11216.0', '11413.0', '10475.0', '10457',
       '11206.0', '10466.0', '11205.0', '10303.0', '11436.0', '10009',
       '10452.0', '11416.0', '11691', '11432', '11219', '10312.0',
       '11201.0', '11213.0', '11374', '11385.0', '11368.0', '10305.0',
       '10473.0', '11226', '11234.0', '11235.0', '11413', '11220.0',
       '11419', '11385', '11233.0', '11236', '10032.0', '11221', '10026',
       '10027.0', '11226.0', '11218.0', '11214.0', '10465.0', '11223.0',
       '10039.0', '11434.0', '11207.0', '11204', '11355.0', '10309.0',
       '10033', '10038.0', '11203', '10034.0', '11230.0', '10025.0',
       '11208', '10031.0', '10472.0', '11236.0', '10013', '11365.0',
       '11104', '11105', '11208.0', '11235', '10462.0', '10002.0',
       '10011.0', '10306', '10035.0', '10035', '10467.0', '11225.0',
       '10453.0', '11435.0', '11212.0', '10016.0', '11239.0', '10021.0',
       '10003', '10456', '11414', '10028.0', '10458.0', '10013.0',
       '11372', '11

#### Standardize the Zipcode using Zfill method as every zipcode is 5 digit long and remove if it contains non-numeric character

In [62]:
df['Incident Zip'] = df['Incident Zip'].apply(lambda x: str(x).zfill(5))
df = df[df['Incident Zip'].str.contains(r'^\d{5}$')]

In [63]:
df['Incident Zip'].unique()

array(['10457', '10009', '11691', '11432', '11219', '11374', '11226',
       '11413', '11419', '11385', '11236', '11221', '10026', '11204',
       '10033', '11203', '11208', '10013', '11104', '11105', '11235',
       '10306', '10035', '10003', '10456', '11414', '11372', '11361',
       '11377', '11218', '10032', '11228', '11423', '11231', '10467',
       '11213', '10314', '11362', '11357', '11435', '10025', '11229',
       '11238', '11369', '11210', '10007', '11370', '10452', '10002',
       '10004', '10451', '11358', '11411', '10039', '10011', '10016',
       '10460', '10461', '11249', '10462', '11217', '11101', '11214',
       '11418', '11355', '10465', '10471', '10301', '11220', '10310',
       '11434', '11233', '11417', '10128', '10453', '11365', '11216',
       '10458', '11375', '10469', '11224', '10031', '11215', '11422',
       '10018', '11230', '10021', '10468', '11367', '10034', '10027',
       '11103', '11222', '11354', '11225', '11106', '11207', '11427',
       '10024', '114

#### City column also has relevance with the target variable. It is important to know from which city we are getting the maximum number of compalaints so that we can resolve that immediately.

In [64]:
df['City'].unique()

array(['BRONX', 'NEW YORK', 'Far Rockaway', 'JAMAICA', 'BROOKLYN',
       'REGO PARK', 'Springfield Gardens', 'SOUTH RICHMOND HILL',
       'RIDGEWOOD', 'SUNNYSIDE', 'ASTORIA', 'STATEN ISLAND', 'Jamaica',
       'Howard Beach', 'JACKSON HEIGHTS', 'BAYSIDE', 'WOODSIDE', 'HOLLIS',
       'Little Neck', 'FAR ROCKAWAY', 'WHITESTONE', 'EAST ELMHURST',
       'Woodside', 'FLUSHING', 'CAMBRIA HEIGHTS', 'South Richmond Hill',
       'LONG ISLAND CITY', 'RICHMOND HILL', 'OZONE PARK', 'Flushing',
       'FRESH MEADOWS', 'Forest Hills', 'Rosedale', 'Bayside', 'Astoria',
       'Queens Village', 'South Ozone Park', 'Long Island City',
       'Sunnyside', 'Ozone Park', 'KEW GARDENS', 'CORONA', 'HOWARD BEACH',
       'Ridgewood', 'Saint Albans', 'Elmhurst', 'SOUTH OZONE PARK',
       'SAINT ALBANS', 'Arverne', 'MASPETH', 'Richmond Hill',
       'FOREST HILLS', 'ELMHURST', 'WOODHAVEN', 'Cambria Heights',
       'Middle Village', 'Glen Oaks', 'ROSEDALE', 'LITTLE NECK',
       'Fresh Meadows', 'COLLEGE

In [65]:
#Bringing to standard form
df['City'] = df['City'].str.strip()
df['City']=df['City'].astype(str).apply(lambda x: x.capitalize())

#Removing rows containing only numeric in city name
df = df[~df['City'].str.isnumeric()]

In [66]:
df['City'].unique()

array(['Bronx', 'New york', 'Far rockaway', 'Jamaica', 'Brooklyn',
       'Rego park', 'Springfield gardens', 'South richmond hill',
       'Ridgewood', 'Sunnyside', 'Astoria', 'Staten island',
       'Howard beach', 'Jackson heights', 'Bayside', 'Woodside', 'Hollis',
       'Little neck', 'Whitestone', 'East elmhurst', 'Flushing',
       'Cambria heights', 'Long island city', 'Richmond hill',
       'Ozone park', 'Fresh meadows', 'Forest hills', 'Rosedale',
       'Queens village', 'South ozone park', 'Kew gardens', 'Corona',
       'Saint albans', 'Elmhurst', 'Arverne', 'Maspeth', 'Woodhaven',
       'Middle village', 'Glen oaks', 'College point', 'Nan', 'Bellerose',
       'Rockaway park', 'Oakland gardens', 'New hyde park', 'Floral park',
       'North bergen', 'Queens', 'Newton', 'Woodmere', 'Valley stream',
       'Hicksville', 'Breezy point', 'Rotkbille centre', 'Newark',
       'Lynwood', 'Broolyn', 'Bklyn', 'Jersey city', 'Port washington',
       'Manhatan', 'Hewlett', 'Centr

####We found that soundex and other fuzzy logics were not efficiently working for few city names such as 'New York' and 'Newark'. It did not recognize that they are different and hence we went ahead with filling it with the most common values with respect to zip codes.

In [69]:
# Multiple cities for single zip code 

new= df[df['Incident Zip'] == '07114']
new[['Incident Zip','City']]

,Incident Zip,City
35570,07114,Newark
82119,07114,New york
191440,07114,Newark
233796,07114,New jersey
286352,07114,Newark
343324,07114,Newark
402737,07114,Newark
412708,07114,Newark
539518,07114,Newark
641452,07114,Newark


In [70]:
# Collecting all possible values for each zip code with cities and their counts

zip_dicti ={}
for index, row in df.iterrows():
    if row['Incident Zip'] in zip_dicti:
        if row['City'] in zip_dicti[row['Incident Zip']]:
            zip_dicti[row['Incident Zip']][row['City']] +=1
        else:
            dicti = {row['City']: 1}
            zip_dicti[row['Incident Zip']].update(dicti)
    else:
        zip_dicti[row['Incident Zip']] = {row['City'] : 1}

print(zip_dicti)

{'10457': {'Bronx': 3401, 'Nan': 2}, '10009': {'New york': 2202, 'Nan': 10}, '11691': {'Far rockaway': 1840, 'Queens': 4}, '11432': {'Jamaica': 2077, 'Bellerose': 1, 'Hollis': 3, 'Flushing': 2}, '11219': {'Brooklyn': 2221, 'Nan': 4}, '11374': {'Rego park': 1369, 'Nan': 8, 'Elmhurst': 3}, '11226': {'Brooklyn': 5073, 'Nan': 7}, '11413': {'Springfield gardens': 1276, 'Rosedale': 1, 'Queens': 7, 'Nan': 3}, '11419': {'South richmond hill': 2140, 'South ozone park': 2, 'Nan': 2}, '11385': {'Ridgewood': 4119, 'Nan': 10, 'Brooklyn': 5, 'Queens': 4, 'Forest hills': 2, 'Kew gardens': 1, 'Ridgewod': 1}, '11236': {'Brooklyn': 2566, 'Nan': 6}, '11221': {'Brooklyn': 3643, 'Nan': 5}, '10026': {'New york': 1576, 'Nan': 6}, '11204': {'Brooklyn': 2277, 'Nan': 5}, '10033': {'New york': 2457, 'Manhatan': 1, 'Nan': 10}, '11203': {'Brooklyn': 2945, 'Nan': 7}, '11208': {'Brooklyn': 3682, 'Queens': 17, 'Nan': 17}, '10013': {'New york': 1644, 'Nan': 4}, '11104': {'Sunnyside': 928, 'Long island city': 2, 'Nan':

In [71]:
# Mapping each zip code with most common used city name

final_zip ={}
for key,value in zip_dicti.items():
    final_zip[key] =max(value, key=value.get)

print(final_zip)

{'10457': 'Bronx', '10009': 'New york', '11691': 'Far rockaway', '11432': 'Jamaica', '11219': 'Brooklyn', '11374': 'Rego park', '11226': 'Brooklyn', '11413': 'Springfield gardens', '11419': 'South richmond hill', '11385': 'Ridgewood', '11236': 'Brooklyn', '11221': 'Brooklyn', '10026': 'New york', '11204': 'Brooklyn', '10033': 'New york', '11203': 'Brooklyn', '11208': 'Brooklyn', '10013': 'New york', '11104': 'Sunnyside', '11105': 'Astoria', '11235': 'Brooklyn', '10306': 'Staten island', '10035': 'New york', '10003': 'New york', '10456': 'Bronx', '11414': 'Howard beach', '11372': 'Jackson heights', '11361': 'Bayside', '11377': 'Woodside', '11218': 'Brooklyn', '10032': 'New york', '11228': 'Brooklyn', '11423': 'Hollis', '11231': 'Brooklyn', '10467': 'Bronx', '11213': 'Brooklyn', '10314': 'Staten island', '11362': 'Little neck', '11357': 'Whitestone', '11435': 'Jamaica', '10025': 'New york', '11229': 'Brooklyn', '11238': 'Brooklyn', '11369': 'East elmhurst', '11210': 'Brooklyn', '10007': 

#### Replacing NAN and less common/wrong values with most common city name for each zip code

In [72]:
for index, row in df.iterrows():
    if row['Incident Zip'] in final_zip:
        if row['City'] != final_zip[row['Incident Zip']]:
            df.at[index, 'City'] = final_zip[row['Incident Zip']]

#### We have total of 5 Borough in NYC. But with Null value it sums up to 6.

In [77]:
df['Borough'].unique() 

array(['BRONX', 'MANHATTAN', 'QUEENS', 'BROOKLYN', nan, 'STATEN ISLAND'],
      dtype=object)

#### Single zip code has mutliple or unknown values or might have wrong values.

In [78]:
new = df[df['Incident Zip'] == '10452']
new['Borough'].unique()

array(['BRONX', nan], dtype=object)

#### We have use nyc_zip_borough.csv master data for checking the Zipcode of our data for the NYC Boroughs with the right one.

We have obtained the list of the Boroughs of NYC and their Zip Codes and we check if a zip code is in a specific NYC Borough. The data was obtained from the following website.
https://data.beta.nyc/en/dataset/pediacities-nyc-neighborhoods/resource/7caac650-d082-4aea-9f9b-3681d568e8a5


In [80]:
#Please change the path for your system
boro_df = pd.read_csv("nyc_zip_borough.csv")

boro_dict ={}
for _, row in boro_df.iterrows():
    boro_dict[str(row['zip'])] = str(row['borough'])

print(boro_dict)

{'10001': 'Manhattan', '10002': 'Manhattan', '10003': 'Manhattan', '10004': 'Manhattan', '10005': 'Manhattan', '10006': 'Manhattan', '10007': 'Manhattan', '10009': 'Manhattan', '10010': 'Manhattan', '10011': 'Manhattan', '10012': 'Manhattan', '10013': 'Manhattan', '10014': 'Manhattan', '10016': 'Manhattan', '10017': 'Manhattan', '10018': 'Manhattan', '10019': 'Manhattan', '10021': 'Manhattan', '10022': 'Manhattan', '10023': 'Manhattan', '10024': 'Manhattan', '10025': 'Manhattan', '10026': 'Manhattan', '10027': 'Manhattan', '10028': 'Manhattan', '10029': 'Manhattan', '10030': 'Manhattan', '10031': 'Manhattan', '10032': 'Manhattan', '10033': 'Manhattan', '10034': 'Manhattan', '10035': 'Manhattan', '10036': 'Manhattan', '10037': 'Manhattan', '10038': 'Manhattan', '10039': 'Manhattan', '10040': 'Manhattan', '10044': 'Manhattan', '10065': 'Manhattan', '10069': 'Manhattan', '10075': 'Manhattan', '10128': 'Manhattan', '10280': 'Manhattan', '10282': 'Manhattan', '10301': 'Staten Island', '1030

#### Update every row with true value of Borough.

In [81]:
for index, row in df.iterrows():
    if row['Incident Zip'] in boro_dict:
            df.at[index, 'Borough'] = boro_dict[row['Incident Zip']].upper()

In [82]:
#result
new= df[df['Incident Zip'] == '10452']
new['Borough'].unique()

array(['BRONX'], dtype=object)

In [83]:
df.to_csv('pre_clustering.csv',encoding ='utf-8')

#### Updating the Incident_address column

In [84]:
df['Incident Address'].unique()

array(['248 EAST  174 STREET', '525 EAST 6 STREET', '14-25 PEARL STREET',
       ..., '1115 63 STREET', '411 EAST 10 STREET',
       '284 WEST 12TH STREET'], dtype=object)

In [85]:
ds = stream("pre_clustering.csv")

##We use clustering from OpenClean to cluster similar values and then update the values in the same cluster with the most common value.


In [86]:
incident_address = ds.update('Incident Address', str.upper).distinct('Incident Address')
clusters = KeyCollision(func=Fingerprint()).clusters(incident_address)

In [87]:
incident_address_dict = {}
cluster_dict = {}
def print_k_clusters(clusters):
    clusters = sorted(clusters, key=lambda x: len(x), reverse=True)
    val_count = sum([len(c) for c in clusters])
    print('Total number of clusters is {} with {} values'.format(len(clusters), val_count))
    for i in range(len(clusters)):
        print('\nCluster {}'.format(i + 1))
        temp_dict = {}
        for key, cnt in clusters[i].items():
          if key == '':
            key = "''"
          print(f'  {key} (x {cnt})')
          temp_dict[key] = int(cnt)
          cluster_dict[key] = i

        incident_address_dict[i] = temp_dict
        
print_k_clusters(clusters)         

Streaming output truncated to the last 5000 lines.

Cluster 3383
  711 EAST  231 STREET (x 10)
  711 EAST 231 STREET (x 1)

Cluster 3384
  966 EAST 231 STREET (x 2)
  966 EAST  231 STREET (x 1)

Cluster 3385
  232 WEST   29 STREET (x 2)
  232 WEST 29 STREET (x 2)

Cluster 3386
  232 WEST   60 STREET (x 1)
  232 WEST 60 STREET (x 1)

Cluster 3387
  664 EAST 232 STREET (x 1)
  664 EAST  232 STREET (x 1)

Cluster 3388
  667 EAST 232 STREET (x 6)
  667 EAST  232 STREET (x 4)

Cluster 3389
  674 EAST 232 STREET (x 1)
  674 EAST  232 STREET (x 1)

Cluster 3390
  857 EAST 232 STREET (x 1)
  857 EAST  232 STREET (x 1)

Cluster 3391
  954 EAST  232 STREET (x 1)
  954 EAST 232 STREET (x 1)

Cluster 3392
  233 EAST 3 STREET (x 1)
  233 EAST    3 STREET (x 1)

Cluster 3393
  233 SOUTH 3 STREET (x 2)
  233 SOUTH    3 STREET (x 1)

Cluster 3394
  233 SOUTH    4 STREET (x 3)
  233 SOUTH 4 STREET (x 4)

Cluster 3395
  233 WEST    4 STREET (x 1)
  233 WEST 4 STREET (x 2)

Cluster 3396
  634 EAST  233 S

In [89]:
# Cluster Number as key and the values in each cluster and their counts as the value
# It is a dictionary of dictionary which will be used for further processing
incident_address_dict 

{0: {'140 WEST  140 STREET': 2,
  '140 WEST 140 STREET': 1,
  '140 WEST STREET': 4,
  'WEST  140 STREET': 1,
  'WEST 140 STREET': 2},
 1: {'144 WEST  144 STREET': 2,
  '144 WEST 144 STREET': 3,
  '144 WEST STREET': 1,
  'WEST  144 STREET': 1,
  'WEST 144 STREET': 2},
 2: {'100 EAST  111 STREET': 1,
  '100 EAST 111 STREET': 1,
  '111 EAST  100 STREET': 12,
  '111 EAST 100 STREET': 4},
 3: {'101 EAST  179 STREET': 1,
  '101 EAST 179 STREET': 2,
  '179 EAST  101 STREET': 1,
  '179 EAST 101 STREET': 1},
 4: {'105 EAST  168 STREET': 2,
  '105 EAST 168 STREET': 2,
  '168 EAST  105 STREET': 1,
  '168 EAST 105 STREET': 1},
 5: {'106 WEST  113 STREET': 1,
  '106 WEST 113 STREET': 3,
  '113 WEST  106 STREET': 1,
  '113 WEST 106 STREET': 1},
 6: {'11 EAST   68 STREET': 1,
  '11 EAST 68 STREET': 1,
  '68 EAST   11 STREET': 1,
  '68 EAST 11 STREET': 1},
 7: {'111 EAST  167 STREET': 4,
  '111 EAST 167 STREET': 2,
  '167 EAST  111 STREET': 1,
  '167 EAST 111 STREET': 2},
 8: {'111 EAST   88 STREET': 

In [91]:
# Most common value in the cluster as key and cluster number as the value.
cluster_dict

{'WEST 140 STREET': 0,
 'WEST  140 STREET': 0,
 '140 WEST STREET': 0,
 '140 WEST  140 STREET': 0,
 '140 WEST 140 STREET': 0,
 'WEST 144 STREET': 1,
 '144 WEST 144 STREET': 1,
 '144 WEST STREET': 1,
 '144 WEST  144 STREET': 1,
 'WEST  144 STREET': 1,
 '111 EAST  100 STREET': 2,
 '100 EAST 111 STREET': 2,
 '111 EAST 100 STREET': 2,
 '100 EAST  111 STREET': 2,
 '101 EAST 179 STREET': 3,
 '101 EAST  179 STREET': 3,
 '179 EAST 101 STREET': 3,
 '179 EAST  101 STREET': 3,
 '105 EAST  168 STREET': 4,
 '168 EAST 105 STREET': 4,
 '105 EAST 168 STREET': 4,
 '168 EAST  105 STREET': 4,
 '106 WEST  113 STREET': 5,
 '113 WEST 106 STREET': 5,
 '106 WEST 113 STREET': 5,
 '113 WEST  106 STREET': 5,
 '68 EAST   11 STREET': 6,
 '68 EAST 11 STREET': 6,
 '11 EAST   68 STREET': 6,
 '11 EAST 68 STREET': 6,
 '111 EAST  167 STREET': 7,
 '167 EAST 111 STREET': 7,
 '111 EAST 167 STREET': 7,
 '167 EAST  111 STREET': 7,
 '111 EAST 88 STREET': 8,
 '111 EAST   88 STREET': 8,
 '88 EAST  111 STREET': 8,
 '88 EAST 111 S

In [92]:
final_addr = {}
for key,value in incident_address_dict.items():
    final_addr[key] = max(value, key=value.get)

print(final_addr)

{0: '140 WEST STREET', 1: '144 WEST 144 STREET', 2: '111 EAST  100 STREET', 3: '101 EAST 179 STREET', 4: '105 EAST  168 STREET', 5: '106 WEST 113 STREET', 6: '68 EAST   11 STREET', 7: '111 EAST  167 STREET', 8: '88 EAST 111 STREET', 9: '125 WEST   55 STREET', 10: '13 WEST 13 STREET', 11: '140 WEST  133 STREET', 12: '136 WEST 170 STREET', 13: 'WEST 30 STREET', 14: 'EAST 35 STREET', 15: 'EAST 36 STREET', 16: 'WEST 40 STREET', 17: 'WEST 42 STREET', 18: 'WEST 44 STREET', 19: 'EAST   57 STREET', 20: 'WEST 95 STREET', 21: '43 EAST 1 STREET', 22: 'WEST    1 STREET', 23: '103 EAST   10 STREET', 24: '25 EAST 10 STREET', 25: 'WEST 10 STREET', 26: '116 EAST  101 STREET', 27: '102 EAST  103 STREET', 28: '14 WEST  103 STREET', 29: '120 WEST 105 STREET', 30: '22 EAST  105 STREET', 31: '105 WEST   55 STREET', 32: 'WEST 105 STREET', 33: '106 WEST   56 STREET', 34: '70 WEST 106 STREET', 35: '107 WEST   68 STREET', 36: '108 WEST 111 STREET', 37: '4 WEST 108 STREET', 38: '109 EAST  153 STREET', 39: '70 W

In [93]:
for index, row in df.iterrows():
    if row['Incident Address'] in cluster_dict:
        df.at[index, 'Incident Address'] = final_addr[cluster_dict[row['Incident Address']]]

#### Updating the StreetName using clustering

In [94]:
street_names = ds.update('Street Name', str.upper).distinct('Street Name')
clusters = KeyCollision(func=Fingerprint(), threads=3).clusters(street_names)

In [95]:
street_name_dict = {}
cluster_dict = {}
def print_k_clusters_and_integrate_sn(clusters):
    clusters = sorted(clusters, key=lambda x: len(x), reverse=True)
    val_count = sum([len(c) for c in clusters])
    print('Total number of clusters is {} with {} values'.format(len(clusters), val_count))
    for i in range(len(clusters)):
        print('\nCluster {}'.format(i + 1))
        temp_dict = {}
        for key, cnt in clusters[i].items():
          if key == '':
            key = "''"
          print(f'  {key} (x {cnt})')

          temp_dict[key] = int(cnt)
          cluster_dict[key] = i
        street_name_dict[i] = temp_dict

print_k_clusters_and_integrate_sn(clusters)

final_street = {}
for key,value in street_name_dict.items():
    final_street[key] = max(value, key=value.get)

Total number of clusters is 728 with 1458 values

Cluster 1
  ST JOHNS AVENUE (x 13)
  AVENUE ST JOHNS (x 2)
  ST JOHN'S AVENUE (x 1)

Cluster 2
  PO BOX (x 2)
  P.O BOX (x 1)
  P.O. BOX (x 1)

Cluster 3
  BRIGHTON 1 PLACE (x 3)
  BRIGHTON    1 PLACE (x 2)

Cluster 4
  BRIGHTON 1 ROAD (x 8)
  BRIGHTON    1 ROAD (x 10)

Cluster 5
  BRIGHTON    1 STREET (x 9)
  BRIGHTON 1 STREET (x 8)

Cluster 6
  BRIGHTON    1 WALK (x 1)
  BRIGHTON 1 WALK (x 1)

Cluster 7
  EAST 1 STREET (x 61)
  EAST    1 STREET (x 37)

Cluster 8
  FLATLANDS 1 STREET (x 3)
  FLATLANDS    1 STREET (x 3)

Cluster 9
  NORTH 1 STREET (x 11)
  NORTH    1 STREET (x 10)

Cluster 10
  PAERDEGAT    1 STREET (x 4)
  PAERDEGAT 1 STREET (x 1)

Cluster 11
  PLUMB 1 STREET (x 6)
  PLUMB    1 STREET (x 1)

Cluster 12
  SOUTH    1 STREET (x 32)
  SOUTH 1 STREET (x 38)

Cluster 13
  WEST    1 STREET (x 32)
  WEST 1 STREET (x 39)

Cluster 14
  BAY   10 STREET (x 8)
  BAY 10 STREET (x 12)

Cluster 15
  BRIGHTON 10 PATH (x 1)
  BRIGHTON  

In [96]:
for index, row in df.iterrows():
    if row['Street Name'] in cluster_dict:
        df.at[index, 'Street Name'] = final_street[cluster_dict[row['Street Name']]]

#### Updating the Cross Street1 column using clustering



In [97]:
cross_street_1 = ds.update('Cross Street 1', str.upper).distinct('Cross Street 1')
clusters = KeyCollision(func=Fingerprint(), threads=3).clusters(cross_street_1)

In [98]:
cross_street_1_dict = {}
cluster_dict = {}

def print_k_clusters_and_integrate_cs1(clusters):
    clusters = sorted(clusters, key=lambda x: len(x), reverse=True)
    val_count = sum([len(c) for c in clusters])
    print('Total number of clusters is {} with {} values'.format(len(clusters), val_count))
    for i in range(len(clusters)):
        print('\nCluster {}'.format(i + 1))
        temp_dict = {}
        for key, cnt in clusters[i].items():
          if key == '':
            key = "''"
          print(f'  {key} (x {cnt})')
          temp_dict[key] = int(cnt)
          cluster_dict[key] = i
        cross_street_1_dict[i] = temp_dict

print_k_clusters_and_integrate_cs1(clusters)

Total number of clusters is 656 with 1319 values

Cluster 1
  '' (x 109851)
  ???????????????????????????? (x 30)
  ???????????????????????????????? (x 1)

Cluster 2
  CROSS BRONX EXPWY ET 1 C-D SB (x 22)
  CROSS BRONX EXPWY SB ET 1 C-D (x 2)
  CROSS BRONX EXPWY ET    1 C-D SB (x 2)

Cluster 3
  CROSS BRONX EXPWY ET 1 D SB (x 19)
  CROSS BRONX EXPWY ET    1 D SB (x 1)
  CROSS BRONX EXPWY SB ET 1 D (x 1)

Cluster 4
  HENRY HUDSON PARKWAY ET 17 NB (x 8)
  HENRY HUDSON PARKWAY ET   17 NB (x 4)
  HENRY HUDSON PARKWAY NB ET 17 (x 1)

Cluster 5
  EAST 179 STREET (x 133)
  EAST  179 STREET (x 7)
  EAST 179 STREET & EAST (x 1)

Cluster 6
  BROOKLYN QUEENS EXPWY ET 33  EB (x 1)
  BROOKLYN QUEENS EXPWY ET   33 EB (x 1)
  BROOKLYN QUEENS EXPWY ET 33 EB (x 3)

Cluster 7
  BROOKLYN QUEENS EXPWY ET 43  WB (x 1)
  BROOKLYN QUEENS EXPWY ET   43 WB (x 1)
  BROOKLYN QUEENS EXPWY ET 43 WB (x 1)

Cluster 8
  CROSS BRONX EXPWY NB ET 1 B (x 9)
  CROSS BRONX EXPWY NB ET    1 B (x 8)

Cluster 9
  BRIGHTON 1 P

In [99]:
final_cs1 = {}
for key,value in cross_street_1_dict.items():
    final_cs1[key] = max(value, key=value.get)

In [100]:
for index, row in df.iterrows():
    if row['Cross Street 1'] in cluster_dict:
        df.at[index, 'Cross Street 1'] = final_cs1[cluster_dict[row['Cross Street 1']]]

#### Updating the Cross Street2 column using clustering

In [101]:
cross_street_2 = ds.update('Cross Street 2', str.upper).distinct('Cross Street 2')
clusters = KeyCollision(func=Fingerprint(), threads=3).clusters(cross_street_2)

In [102]:
cross_street_2_dict = {}
cluster_dict = {}
def print_k_clusters_and_integrate_cs2(clusters):
    clusters = sorted(clusters, key=lambda x: len(x), reverse=True)
    val_count = sum([len(c) for c in clusters])
    print('Total number of clusters is {} with {} values'.format(len(clusters), val_count))
    for i in range(len(clusters)):
        print('\nCluster {}'.format(i + 1))
        temp_dict = {}
        for key, cnt in clusters[i].items():
          if key == '':
            key = "''"
          print(f'  {key} (x {cnt})')

          temp_dict[key] = int(cnt)
          cluster_dict[key] = i
        cross_street_2_dict[i] = temp_dict

print_k_clusters_and_integrate_cs2(clusters)

Total number of clusters is 673 with 1351 values

Cluster 1
  LONG ISLAND EXPWY ET   19 WB (x 1)
  LONG ISLAND EXPWY ET 19 WB (x 3)
  LONG ISLAND EXPWY ET 19  WB (x 1)

Cluster 2
  HARLEM RIVER DRIVE EXIT 21 NB (x 13)
  HARLEM RIVER DRIVE EXIT   21 NB (x 1)
  HARLEM RIVER DRIVE NB EXIT 21 (x 3)

Cluster 3
  GOWANUS EXPRESSWAY EXIT 22 WB (x 26)
  GOWANUS EXPRESSWAY WB EXIT 22 (x 4)
  GOWANUS EXPRESSWAY EXIT   22 WB (x 1)

Cluster 4
  HARLEM RIVER DRIVE EXIT 23 SB (x 18)
  HARLEM RIVER DRIVE EXIT   23 SB (x 2)
  HARLEM RIVER DRIVE EXIT 23  SB (x 1)

Cluster 5
  CROSS BRONX EXPWY ET 4 A  EB (x 1)
  CROSS BRONX EXPWY ET 4 A   EB (x 1)
  CROSS BRONX EXPWY ET 4 A EB (x 1)

Cluster 6
  '' (x 110144)
  ???????????????????????????? (x 13)

Cluster 7
  CROSS BRONX EXPWY NB ET 1 A (x 18)
  CROSS BRONX EXPWY NB ET    1 A (x 1)

Cluster 8
  CROSS BRONX EXPWY SB ET    1 A (x 1)
  CROSS BRONX EXPWY SB ET 1 A (x 5)

Cluster 9
  CROSS BRONX EXPWY NB ET 1 B (x 3)
  CROSS BRONX EXPWY NB ET    1 B (x 1)



In [103]:
final_cs2 = {}
for key,value in cross_street_2_dict.items():
    final_cs2[key] = max(value, key=value.get)

In [104]:
for index, row in df.iterrows():
    if row['Cross Street 2'] in cluster_dict:
        df.at[index, 'Cross Street 2'] = final_cs2[cluster_dict[row['Cross Street 2']]]

In [105]:
df[['Latitude','Longitude']].describe(percentiles=[.01,.05,.1,.25,.5,.9,.95,.99])
# since there is no large variance in bins there are no outlier's

,Latitude,Longitude
count,288024.000000,288024.000000
mean,40.728834,-73.924603
std,0.085836,0.082476
min,40.498949,-74.254437
1%,40.550832,-74.169787
5%,40.595663,-74.076645
10%,40.617444,-74.002017
25%,40.666924,-73.969975
50%,40.721681,-73.927999
90%,40.851418,-73.818856


In [106]:
df.isnull().values.any()

True

In [107]:
df.dtypes

Created Date                       object
Closed Date                        object
Agency                             object
Agency Name                        object
Complaint Type                     object
Descriptor                         object
Location Type                      object
Incident Zip                       object
Incident Address                   object
Street Name                        object
Cross Street 1                     object
Cross Street 2                     object
Address Type                       object
City                               object
Status                             object
Resolution Description             object
Resolution Action Updated Date     object
Community Board                    object
BBL                               float64
Borough                            object
X Coordinate (State Plane)        float64
Y Coordinate (State Plane)        float64
Open Data Channel Type             object
Park Borough                      

#### Finalizing the columns: Location Type, Address Type, Status, Resolution Description, Community Board, BBL, X-Cord, Y-Cord, Open Data Channel Type, Park Borough, Location

In [108]:
df['Latitude'].isnull().sum()

20578

In [109]:
df['Latitude'].unique()

array([40.84469154, 40.72516461, 40.60978147, ..., 40.55994995,
       40.62947745, 40.73740737])

#### We have null values in Latitude column, but we can't impute it because Latitude is different for every location.

In [110]:
df = df[df['Latitude'].notna()]

In [111]:
df['Latitude'].isnull().sum()

0

In [112]:
df['Longitude'].isnull().sum()

0

In [113]:
df['Location'].isnull().sum()

0

In [114]:
df['Location Type'].isnull().sum()

63990

#### If we look at unique values in the 'Location Type' column, we'll see that some values are in title case while others are in upper case.

In [115]:
df['Location Type'].unique()

array([nan, 'Residential Building/House', 'Restaurant/Bar/Deli/Bakery',
       'Street/Sidewalk', 'RESIDENTIAL BUILDING', 'Street', 'Home',
       'Other (Explain Below)', 'Sidewalk', 'Comercial',
       'Government Building', '3+ Family Apt. Building', 'Mixed Use',
       'Recreation Center', 'Park', 'Store/Commercial',
       '1-2 Family Dwelling', 'Club/Bar/Restaurant', 'Curb',
       'Street and Sidewalk', 'Commercial Building', 'Park/Playground',
       'Other', 'Bridge', 'Residential', 'Lot',
       '3+ Family Apartment Building', 'Public Garden', 'Above Address',
       'Street/Curbside', 'Vacant Building', 'Street Address',
       'Speed Reducer', 'Residence', 'Vacant Lot', 'School', 'Pet Shop',
       'Property Address', 'Public Garden/Park', 'Highway',
       'Single Room Occupancy (SRO)', 'Parking Lot/Garage',
       '3+ Family Mixed Use Building', 'Bus Stop Shelter',
       'Residential Building', 'Crosswalk', 'School/Pre-School',
       'Public School', 'Public/Unfenced Ar

#### Below, all of the values in the column are converted to title case.
We also replace the null values with the 'Unrecognized'

In [152]:
df['Location Type'] = df['Location Type'].str.title()
df['Location Type'] = df['Location Type'].replace(np.nan, 'Unrecognized')

In [153]:
df['Location Type'].isnull().sum()

0

#### Address Type column has Null values but we can't impute/replace it with some another value as it is a string value. So we replace the null values with 'UNRECOGNIZED'.

In [118]:
df['Address Type'].isnull().sum()

9765

In [119]:
df['Address Type'].unique()

array(['ADDRESS', nan, 'INTERSECTION', 'BLOCKFACE', 'LATLONG',
       'PLACENAME', 'UNRECOGNIZED'], dtype=object)

In [150]:
df['Address Type'] = df['Address Type'].replace(np.nan, 'UNRECOGNIZED')

In [151]:
df['Address Type'].isnull().sum()

0

In [122]:
df['Status'].isnull().sum()

0

#### Resolution Description has some rows without any reasons provided. There might be a chance that officer forgot to register it. We can not impute it with any other random description because description is unique for all the complaints.

In [123]:
df['Resolution Description'].isnull().sum()

5922

In [124]:
df['Resolution Description'].unique()

array(['The Police Department responded to the complaint and took action to fix the condition.',
       'The Department of Health and Mental Hygiene has received and processed your complaint.  All restaurants and mobile food vendors are inspected annually.  Restaurant inspection results can be found on WWW.NYC.GOV or a copy of the inspection can be requested from 311.',
       'The Department of Housing Preservation and Development inspected the following conditions. No violations were issued. The complaint has been closed.',
       'The complaint you filed is a duplicate of a condition already reported by another tenant for a building-wide condition. The original complaint is still open. HPD may attempt to contact you to verify the correction of the condition or may conduct an inspection of your unit if the original complainant is not available for verification.',
       'The agency has declined the new tree request because the suggested location cannot be planted due to infrastructur

In [125]:
df = df[df['Resolution Description'].notna()]

In [126]:
df['Resolution Description'].isnull().sum()

0

In [127]:
df['Community Board'].isnull().sum()

0

In [128]:
df['BBL'].isnull().sum()

30882

In [129]:
df['BBL'].unique()

array([1.00402005e+09, 4.09854000e+09, 4.02093024e+09, ...,
       5.05683013e+09, 3.05731006e+09, 4.00611001e+09])

In [130]:
df['BBL'].isnull().sum()

30882

In [131]:
df['X Coordinate (State Plane)'].isnull().sum()

0

In [132]:
df['Y Coordinate (State Plane)'].isnull().sum()

0

In [133]:
df['Open Data Channel Type'].isnull().sum()

0

In [134]:
df['Park Borough'].isnull().sum()

2049

## We copy the same values of Borough in Park Borough

In [141]:
df['Park Borough'] = df['Borough']

In [142]:
df['Park Borough'].unique()

array(['MANHATTAN', 'QUEENS', 'BROOKLYN', 'STATEN ISLAND', 'BRONX'],
      dtype=object)

In [143]:
df['Park Borough'].isnull().sum()

0

In [144]:
df['Location'].isnull().sum()

0

In [146]:
df['Complaint Type'].unique()

array(['Noise - residential', 'Food establishment', 'Blocked driveway',
       'Water leak', 'Heat/hot water', 'New tree request',
       'Paint/plaster', 'Street condition', 'Noise - street/sidewalk',
       'Noise survey', 'Root/sewer/sidewalk condition', 'General',
       'Missed collection (all materials)', 'Dead tree',
       'Illegal parking', 'Unsanitary condition', 'Electric', 'Graffiti',
       'Smoking', 'Rodent', 'Sidewalk condition', 'Flooring/stairs',
       'Heating', 'Vending', 'Electronics waste', 'Derelict vehicle',
       'Maintenance or facility', 'Overgrown tree/branches',
       'Noise - commercial', 'Broken muni meter', 'Standing water',
       'Taxi complaint', 'Damaged tree', 'Noise - vehicle',
       'Other enforcement', 'Plumbing', 'Safety', 'Derelict bicycle',
       'Curb condition', 'Public payphone complaint',
       'Sanitation condition', 'Traffic', 'Door/window', 'Noise - park',
       'Dirty conditions', 'Street sign - dangling', 'Drinking',
       'St

In [147]:
df.shape

(206298, 27)

##We again run remove duplicates as we may have changed some values and hence obtained some duplicates in the process.

In [148]:
df = df.drop_duplicates()

In [149]:
df.shape

(205920, 27)

## With this we conclude our cleaning process and we are ready to perform analysis on the cleaned data.